In [ ]:
!pip install optuna
!pip install catboost

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split, GridSearchCV
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
train = pd.read_csv ("diabetes.csv")
test = pd.read_csv("diabetes_test_unlabeled.csv")

In [4]:
remove = ['encounter_id' , 'hospital_id' , 'd1_row_nan_counts' , 'h1_row_nan_counts' , 'nan_counts']

In [6]:
num = []
cat = []

def encode_categorical_features(train, test):
    for col in train.columns:   
        if train[col].dtype == 'object':
            if train[col].nunique() <= 15:
                encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
                train_encoded = encoder.fit_transform(train[[col]])
                test_encoded = encoder.transform(test[[col]])
                train.drop(columns=[col], inplace=True)
                test.drop(columns=[col], inplace=True)
                train = pd.concat([train, pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([col]))], axis=1)
                test = pd.concat([test, pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([col]))], axis=1)
            else:
                encoder = LabelEncoder()
                train_encoded = encoder.fit_transform(train[col])
                test_encoded = encoder.transform(test[col])
                train[col] = train_encoded
                test[col] = test_encoded
        else:
            num.append(col)
    return train, test


train , test = encode_categorical_features(train , test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

In [7]:
for i in train:
    if i not in num:
        cat.append(i)

In [9]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def optimize_hyperparameters(X_train, y_train, n_trials=100):
    def objective(trial):
        params = {
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'n_estimators': trial.suggest_int('n_estimators', 500, 2000)
        }


        params['task_type'] = "GPU"
        params['verbose'] = 0


        model = CatBoostRegressor()
        model.set_params(**params)

        X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2)

        model.fit(X_train_split, y_train_split)

        y_pred = model.predict(X_val)

        mse = mean_squared_error(y_val, y_pred)

        return mse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_trial.params
    best_params['task_type'] = "GPU"
    best_params['verbose'] = 0
    model = CatBoostRegressor()
    model.set_params(**best_params)

    return model , best_params

In [10]:
import pandas as pd
from catboost import CatBoostRegressor

df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])
df = df.drop(remove,axis=1)
params = {'albumin_apache': {'learning_rate': 0.047669878474493055, 'max_depth': 5, 'n_estimators': 1973, 'task_type': 'GPU', 'verbose': 0}, 'bilirubin_apache': {'learning_rate': 0.01568689850000682, 'max_depth': 5, 'n_estimators': 1498, 'task_type': 'GPU', 'verbose': 0}, 'bun_apache': {'learning_rate': 0.012212797074912045, 'max_depth': 3, 'n_estimators': 1727, 'task_type': 'GPU', 'verbose': 0}, 'creatinine_apache': {'learning_rate': 0.045852459281106775, 'max_depth': 9, 'n_estimators': 1285, 'task_type': 'GPU', 'verbose': 0}, 'hematocrit_apache': {'learning_rate': 0.08238165195172158, 'max_depth': 8, 'n_estimators': 1996, 'task_type': 'GPU', 'verbose': 0}, 'sodium_apache': {'learning_rate': 0.05962428465017329, 'max_depth': 5, 'n_estimators': 750, 'task_type': 'GPU', 'verbose': 0}, 'urineoutput_apache': {'learning_rate': 0.008872279046660136, 'max_depth': 5, 'n_estimators': 1599, 'task_type': 'GPU', 'verbose': 0}, 'wbc_apache': {'learning_rate': 0.04820822552989037, 'max_depth': 3, 'n_estimators': 1286, 'task_type': 'GPU', 'verbose': 0}, 'h1_temp_max': {'learning_rate': 0.012806129173317955, 'max_depth': 5, 'n_estimators': 1368, 'task_type': 'GPU', 'verbose': 0}, 'h1_temp_min': {'learning_rate': 0.049510781193983146, 'max_depth': 5, 'n_estimators': 982, 'task_type': 'GPU', 'verbose': 0}, 'd1_albumin_max': {'learning_rate': 0.046289929379345916, 'max_depth': 3, 'n_estimators': 685, 'task_type': 'GPU', 'verbose': 0}, 'd1_albumin_min': {'learning_rate': 0.024870142473933966, 'max_depth': 7, 'n_estimators': 1178, 'task_type': 'GPU', 'verbose': 0}, 'd1_bilirubin_max': {'learning_rate': 0.08242483751582927, 'max_depth': 7, 'n_estimators': 1507, 'task_type': 'GPU', 'verbose': 0}, 'd1_bilirubin_min': {'learning_rate': 0.04884534873218641, 'max_depth': 3, 'n_estimators': 1879, 'task_type': 'GPU', 'verbose': 0}, 'd1_hco3_max': {'learning_rate': 0.011021670127736352, 'max_depth': 7, 'n_estimators': 1525, 'task_type': 'GPU', 'verbose': 0}, 'd1_hco3_min': {'learning_rate': 0.051470605840941513, 'max_depth': 5, 'n_estimators': 992, 'task_type': 'GPU', 'verbose': 0}, 'd1_inr_max': {'learning_rate': 0.05412116272722388, 'max_depth': 3, 'n_estimators': 692, 'task_type': 'GPU', 'verbose': 0}, 'd1_inr_min': {'learning_rate': 0.09813297931498835, 'max_depth': 4, 'n_estimators': 1929, 'task_type': 'GPU', 'verbose': 0}, 'h1_glucose_max': {'learning_rate': 0.08179184254899058, 'max_depth': 5, 'n_estimators': 1053, 'task_type': 'GPU', 'verbose': 0}, 'h1_glucose_min': {'learning_rate': 0.06344391201411274, 'max_depth': 4, 'n_estimators': 1206, 'task_type': 'GPU', 'verbose': 0}, 'd1_arterial_pco2_max': {'learning_rate': 0.05055896951597059, 'max_depth': 4, 'n_estimators': 952, 'task_type': 'GPU', 'verbose': 0}, 'd1_arterial_pco2_min': {'learning_rate': 0.048839365196095295, 'max_depth': 5, 'n_estimators': 591, 'task_type': 'GPU', 'verbose': 0}, 'd1_arterial_ph_max': {'learning_rate': 0.020551684368551114, 'max_depth': 5, 'n_estimators': 1921, 'task_type': 'GPU', 'verbose': 0}, 'd1_arterial_ph_min': {'learning_rate': 0.03549253031513147, 'max_depth': 4, 'n_estimators': 1489, 'task_type': 'GPU', 'verbose': 0}, 'd1_arterial_po2_max': {'learning_rate': 0.016888444177358092, 'max_depth': 5, 'n_estimators': 1766, 'task_type': 'GPU', 'verbose': 0}, 'd1_arterial_po2_min': {'learning_rate': 0.050409971791460936, 'max_depth': 3, 'n_estimators': 1864, 'task_type': 'GPU', 'verbose': 0}}
from catboost import CatBoostRegressor

def fill_nulls_with_catboost(df, target_column):
    train_df = df[df[target_column].notnull()]
    test_df = df[df[target_column].isnull()]

    excluded_columns = ['ind', 'diabetes_mellitus']
    feature_columns = [col for col in df.columns if col not in excluded_columns]
    X_train = train_df[feature_columns]
    y_train = train_df[target_column]
    X_test = test_df[feature_columns]

    #model , params = optimize_hyperparameters(X_train,y_train,10)
    model = CatBoostRegressor(**params[target_column])
    model.fit(X_train, y_train)

    predicted_values = model.predict(X_test)

    df.loc[df[target_column].isnull(), target_column] = predicted_values

    return df

In [11]:
nulls = []
def null_info(df):
    num_rows = len(df)
    
    for col in df.columns:
        num_nulls = df[col].isnull().sum()
        
        if num_nulls > 0:
            null_percent = round((num_nulls / num_rows) * 100, 2)
            if(null_percent) > 15:
                nulls .append(col)
            
            #print(f"Column '{col}' has {num_nulls} null values ({null_percent}% of total)")
            
            
            
null_info(df)

def fill_mean(df):
    for col in df.columns:
        if df[col].isnull().any():
            col_mean = df[col].mean()
            df[col].fillna(col_mean, inplace=True)

nulls.remove('diabetes_mellitus')

In [12]:
for i in nulls:
  df = fill_nulls_with_catboost(df, i)

In [13]:
fill_mean(df)


In [14]:
test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]

to_drop = ['ind' , 'diabetes_mellitus']
label = ['diabetes_mellitus']


X = train.drop(to_drop,axis=1)
Y = train[label]


test = test.drop(to_drop,axis=1)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)
test = scaler.transform(test)

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.33, random_state=42)


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# Define the model
model = Sequential()

model.add(Dense(256, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               40448     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 10)               

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))


Epoch 1/20
1953/1953 [==============================] - 12s 5ms/step - loss: 0.4502 - accuracy: 0.8089 - val_loss: 0.4141 - val_accuracy: 0.8189
Epoch 2/20
1953/1953 [==============================] - 11s 6ms/step - loss: 0.4085 - accuracy: 0.8216 - val_loss: 0.4008 - val_accuracy: 0.8184
Epoch 3/20
1953/1953 [==============================] - 9s 5ms/step - loss: 0.3996 - accuracy: 0.8220 - val_loss: 0.3926 - val_accuracy: 0.8219
Epoch 4/20
1953/1953 [==============================] - 10s 5ms/step - loss: 0.3940 - accuracy: 0.8253 - val_loss: 0.3926 - val_accuracy: 0.8218
Epoch 5/20
1953/1953 [==============================] - 12s 6ms/step - loss: 0.3900 - accuracy: 0.8266 - val_loss: 0.3911 - val_accuracy: 0.8237
Epoch 6/20
1953/1953 [==============================] - 10s 5ms/step - loss: 0.3881 - accuracy: 0.8253 - val_loss: 0.3869 - val_accuracy: 0.8248
Epoch 7/20
1953/1953 [==============================] - 10s 5ms/step - loss: 0.3848 - accuracy: 0.8281 - val_loss: 0.3848 - val_acc

In [26]:
predictions = model.predict(test)

threshold = 0.5

binary_predictions = [1 if pred >= 0.5 else 0 for pred in predictions]

1627/1627 [==============================] - 3s 2ms/step


In [27]:
sub = pd.DataFrame({label[0] : np.array(binary_predictions)} , index = pd.read_csv('Sample_Submission.csv')['ID'])
sub.index = sub.index.rename('ID')
sub

,diabetes_mellitus
ID,
172759,0
196306,0
249171,0
161332,0
232215,0
...,...
185025,0
169719,0
158245,1


In [28]:
sub[label].value_counts()

diabetes_mellitus
0                    44080
1                     7983
dtype: int64

In [ ]:
model.save('model#1')